# Chapter 4: Aggregate Functions

[**4.1 Aggregation**](#4.1-Aggregation)   
[**4.2 Aggregate Functions**](#4.2-Aggregate-Functions)   
[**4.3 Grouping Types**](#4.3-Grouping-Types)   
[**4.3.1 Simple Grouping**](#4.3.1-Simple-Grouping)   
[**4.3.2 GroupBy**](#4.3.2-GroupBy)   
[**4.3.3 Window**](#4.3.3-Window)   
[**4.3.4 Grouping Set**](#4.3.4-Grouping-Set)   
[**4.3.4.1 Rollup**](#4.3.4.1-Rollup)   
[**4.3.4.2 Cube**](#4.3.4.2-Cube)   
[**4.4 Pivot**](#4.4-Pivot)   
[**4.5 User Defined Aggregate Functions**](#4.5-User-Defined-Aggregate-Functions)

#### 4.1 Aggregation
**Aggregation**: Aggregation is the process of summarizing the data based on one or more columns. Aggregated data provides the insights of particular groups. Aggregation is core functionality of data analytics. Data can be aggregated either by whole dataset or grouping into individual/groups of attributes.

Some examples of aggregation are:      
* Counting total record.
* Counting record based specified column. For e.g. counting record based on state, zipcode, gender etc.
* Finding minimum and maximum sales for given dataset.
* Finding sum of total sales.
* Finding average score in SAT.
* Finding total sales based on year and month.
* Finding total customer served each day based on time duration. e.g. grouped on 6-9, 9-11, 11-2 etc.
* Finding mean, standard deviation, variance, correlation, skew etc.

#### 4.2 Aggregate Functions
Spark has many built-in aggregate functions. All the functions can be imported from `pyspark.sql.functions`. Table 4.2 shows aggregate functions for simple and complex types in Spark. 

Table 4.2 Spark aggregate functions   

| Function Name | Description | Function with Parameter |
| ------------- | ------------- | ------------- |
| approx_count_distinct | Returns the approximate number of distinct items in a group | approx_count_distinct(column_name) |
| avg | Return the average values for each numeric columns for each group. Similar to `mean`. Non-numerical columns will be ignored. | avg(column_name) or avg(\*column_name) | 
| collect_list | Returns a list of objects with duplicates. | collect_list(column_name) |
| collect_set | Returns a set of objects by eliminating duplicates. | collect_set(column_name) |
| corr | Returns the Pearson Correlation Coefficient for given columns (column 1 and column 2) | cor(column_1, column_2) |
| count | Returns the number of items in a group. | count(column_name) |
| countDistinct | Returns the distinct count for either column or list of colummns. | countDistinct(column_name) or countDistinct(\*column_name) |
| covar_pop | Returns the population covariance of column_1 and column_2. | covar_pop(column_name_1, column_name_2) |
| covar_samp | Returns the sample covariance of column_1 and column_2. | covar_sample(column_name_1, column_name_2) |
| first | Returns the first row of column. By default it will return the first values it finds. When ignoreNull is set to true then it will return first non-null values. If all vaues are null then null values will be returned. | first(column_name)  or first(column_name, ignorenulls = Boolean)
| grouping | Indicate if a specified column in a GROUP BY list is aggregated or not, Returns 1 for aggregated or 0 or not aggregated. | grouping(column_name) |
| grouping_id | Returns the level of grouping. i.e. (grouping(c1)<<(n-1)) + (grouping(c1)<<(n-1)) + ... + (grouping(cn)<<(cn)). |
| kurtosis | Returns the kurtosis of the column. | kurtosis(column_name) |
| last | Returns the last row of column. By default it will return the last values it finds. When ignoreNull is set to true then it will return last non-null values. If all vaues are null then null values will be returned.  | last(column_name) or last(column_name, ignorenulls = Boolean) | 
| max | Returns the maximum values of the column in a group. | max(column_name) |
| mean | Returns the average values of the column in a group. | mean(column_name) |
| min | Returns the minimun values of the column in a group | min(column_name) |
| skewness | Returns the skewness of the values in a group | skewness(column_name) |
| stddev | Returns the sample standard deviation of expression in a group. Similar to `stddev_samp`. | stddev(column_name) |
| stddev_pop | Returns the population standard deviation of expression in a group.  | stddev_pop(column_name) |
| stddev_samp | Returns the sample standard deviation of expression in a group. | stddev_samp(column_name) |
| sum | Returns the sum of all values in a column or expression. | sum(column_name) or sum(\*column_name) |
| sumDistinct | Returns the sum of of distinct values in a column or expression. | sumDistinct(column_name) or sumDistinct(column_expression) |
| var_pop | Returns the population variance of the values in a group | var_pop(column_name) |
| var_sample | Returns the unbiased sample variance of the values in a group | var_sample(column_name) |
| variance | Returns the unbiased sample variance of the values in a group. Similar to `var_sample`. | variance(column_name) |   

$^{*}$indicate multiple column name.

#### 4.3 Grouping Types
Spark provides several grouping types for aggregation. The grouping types are:
*  `simple grouping`: is used to get summary of DataFrame based on specific columns. The aggregate function is used in `select` statement.
* `groupby`: is used to specify one or more columns in aggregate functions. It also allows to specify one or more aggregate functions for specific columns.
* `window`: is used to specify one or more columns in aggregate functions whereas the input to the functions are related to current row.
* `grouping set`: is used to aggregate across multiple groups. `rollup` and `cube` are used for grouping set.
    * `rollup`: is used to aggregate over combination of groups of specified columns.
    * `cube`: is used to aggregate across every permutation of specified columns. i.e. across all combination of specified columns.

**Prerequisites: Read all the tables from employees database in MySQL into Spark.**

In [1]:
import configparser

# Read mysql database connection string from conf/db_properties.ini

config_filename = '../Chapter_2_Structured_API/Lab_1/conf/db_properties.ini'
db_properties = {}
config = configparser.ConfigParser()
config.read(config_filename)
db_prop = config['mysql']
db_url = db_prop['url']
db_properties['database'] = db_prop['database']
db_properties['schema'] = db_prop['schema']
db_properties['user'] = db_prop['user']
db_properties['password'] = db_prop['password']
db_properties['serverTimezone'] = db_prop['serverTimezone']

In [2]:
from pyspark.sql import SparkSession

# Create Spark Session
spark = SparkSession.builder \
    .master("local") \
    .appName("Chapter 4 Aggregate Function") \
    .getOrCreate()

# Load current_dept_emp
current_dept_emp = spark.read.jdbc(url = db_url, table = 'current_dept_emp', properties = db_properties)

# Load departments
departments = spark.read.jdbc(url = db_url, table = 'departments', properties = db_properties)

# Load dept_emp
dept_emp = spark.read.jdbc(url = db_url, table = 'dept_emp', properties = db_properties)

# Load dept_emp_latest_date
dept_emp_latest_date = spark.read.jdbc(url = db_url, table = 'dept_emp_latest_date', properties = db_properties)

# Load dept_manager
dept_manager = spark.read.jdbc(url = db_url, table = 'dept_manager', properties = db_properties)

# Load employees
employees = spark.read.jdbc(url = db_url, table = 'employees', properties = db_properties)

# Load highest_salary_employee
highest_salary_employee = spark.read.jdbc(url = db_url, table = 'highest_salary_employee', properties = db_properties)

# Load salaries
salaries = spark.read.jdbc(url = db_url, table = 'salaries', properties = db_properties)

# Load titles
titles = spark.read.jdbc(url = db_url, table = 'titles', properties = db_properties)

In [3]:
# Show Spark Session
spark

# Show employees DF schema 
employees.printSchema()

root
 |-- emp_no: integer (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- hire_date: date (nullable = true)



#### 4.3.1 Simple Grouping

Aggregate function are used in `select` statement. We'll apply aggregate function shown in table 4.2.

**count**   
`count`: It is used for counting total records in DataFrame. Records can be counted either by passing \* or column name to `count()` method as shown below:      
* count("column_name"): If the column contains null value then those will be ignored.
* count("\*"): If the column contains null value then those will be counted.
* count(lit(1)): We can also pass the literal value to count each record.

We can also perform basic aggregation over entire DataFrame by specifying `df.method_name()`. For example:   
`employees.count()`. It is the quickest method.

In [4]:
# count over entire DataFrame
employees.count()

300024

In [5]:
# count all the records from employees DF.

from pyspark.sql.functions import count

employees.select(count("*").alias("total_records_in_employees_DF")).show()

+-----------------------------+
|total_records_in_employees_DF|
+-----------------------------+
|                       300024|
+-----------------------------+



In [6]:
employees.select(count("*").alias("total_count"))

DataFrame[total_count: bigint]

In [7]:
# count all the records from employees DF.
# Specify only emp_no

employees.select(count("emp_no").alias("total_records_in_employees_DF")).show()

+-----------------------------+
|total_records_in_employees_DF|
+-----------------------------+
|                       300024|
+-----------------------------+



In [8]:
# count all the records from employees DF.
# with literal value

from pyspark.sql.functions import lit

employees.select(count(lit(1)).alias("total_records_in_employees_DF")).show()

+-----------------------------+
|total_records_in_employees_DF|
+-----------------------------+
|                       300024|
+-----------------------------+



**Create emp DataFrame**:   
Since we don't have DataFrame with null values, we will create emp DataFrame from employees by adding two new column new `isabove60` and `age_above_60` with boolean and string type respectively. `age_above_60` will have  only two value either `Yes` or `null`. This helps us to compare DataFrame with and without null values in aggregate functions.

In [9]:
# Check if first columns value is empty or null, if it is empty then retrieve value from second column or put literal value.

from pyspark.sql.functions import col, expr, coalesce, current_date, floor, months_between

# Create new DataFrame tmpDF that contain all the columns from employees and adding new columns "age_above_50"
# column 'age_above_60' is calculated field that stores whether employees age is above 60 or not.

# assign value
total_month = 12
filter_age = 60

# emp_current_age stores employee current age
# isabove50 stores boolean value whether or not age is 50
# age_above_60 stores Yes for above age 60 else null

emp = employees.withColumn("emp_current_age",\
                  floor(months_between(current_date(), "birth_date")/total_month))\
                  .withColumn("isabove60", col("emp_current_age") >= filter_age)\
                  .withColumn("age_above_60", expr("case when isabove60 then 'Yes' else null end"))                             
emp.show(20)

+------+----------+----------+-----------+------+----------+---------------+---------+------------+
|emp_no|birth_date|first_name|  last_name|gender| hire_date|emp_current_age|isabove60|age_above_60|
+------+----------+----------+-----------+------+----------+---------------+---------+------------+
| 10001|1953-09-02|    Georgi|    Facello|     M|1986-06-26|             67|     true|         Yes|
| 10002|1964-06-02|   Bezalel|     Simmel|     F|1985-11-21|             56|    false|        null|
| 10003|1959-12-03|     Parto|    Bamford|     M|1986-08-28|             60|     true|         Yes|
| 10004|1954-05-01| Chirstian|    Koblick|     M|1986-12-01|             66|     true|         Yes|
| 10005|1955-01-21|   Kyoichi|   Maliniak|     M|1989-09-12|             65|     true|         Yes|
| 10006|1953-04-20|    Anneke|    Preusig|     F|1989-06-02|             67|     true|         Yes|
| 10007|1957-05-23|   Tzvetan|  Zielinski|     F|1989-02-10|             63|     true|         Yes|


Applying the count method in emp DataFrame to see the effect that has null values.

In [10]:
emp.count()

300024

In [11]:
emp.select(count("*").alias("total_records_in_employees_DF")).show()
emp.select(count("emp_no").alias("total_records_in_employees_DF")).show()

# Count records in age_above_60 column that contains null value
emp.select(count("age_above_60").alias("total_records_in_employees_DF")).show()

+-----------------------------+
|total_records_in_employees_DF|
+-----------------------------+
|                       300024|
+-----------------------------+

+-----------------------------+
|total_records_in_employees_DF|
+-----------------------------+
|                       300024|
+-----------------------------+

+-----------------------------+
|total_records_in_employees_DF|
+-----------------------------+
|                       203161|
+-----------------------------+



**countDistinct**

`countDistinct`: It is used to count total distinct records in DataFrame. Basically, counting records excluding duplicate values. It is used mainly in specific columns instead in entire DataFrame.

In [12]:
 # count distinct gender in employees DF

from pyspark.sql.functions import countDistinct

employees.select(countDistinct("gender").alias("Distinct Gender")).show()

+---------------+
|Distinct Gender|
+---------------+
|              2|
+---------------+



In [13]:
 # count distinct first name in employees DF

from pyspark.sql.functions import countDistinct

employees.select(countDistinct("first_name").alias("Distinct FirstName")).show()

+------------------+
|Distinct FirstName|
+------------------+
|              1275|
+------------------+



In [14]:
 # count distinct last name in employees DF

from pyspark.sql.functions import countDistinct

employees.select(countDistinct("last_name").alias("Distinct LastName")).show()

+-----------------+
|Distinct LastName|
+-----------------+
|             1637|
+-----------------+



In [15]:
 # count distinct fullname in employees DF

from pyspark.sql.functions import countDistinct, concat

employees.select(countDistinct((concat("first_name", "last_name"))).alias("Distinct FullName")).show()

+-----------------+
|Distinct FullName|
+-----------------+
|           279408|
+-----------------+



`dept_emp` DataFrame stores all the department served by each employee on given timeframe. We'll find total dept_no in that DataFrame.

In [16]:
dept_emp.printSchema()    
dept_emp.show(10)

root
 |-- emp_no: integer (nullable = true)
 |-- dept_no: string (nullable = true)
 |-- from_date: date (nullable = true)
 |-- to_date: date (nullable = true)

+------+-------+----------+----------+
|emp_no|dept_no| from_date|   to_date|
+------+-------+----------+----------+
| 10001|   d005|1986-06-26|9999-01-01|
| 10002|   d007|1996-08-03|9999-01-01|
| 10003|   d004|1995-12-03|9999-01-01|
| 10004|   d004|1986-12-01|9999-01-01|
| 10005|   d003|1989-09-12|9999-01-01|
| 10006|   d005|1990-08-05|9999-01-01|
| 10007|   d008|1989-02-10|9999-01-01|
| 10008|   d005|1998-03-11|2000-07-31|
| 10009|   d006|1985-02-18|9999-01-01|
| 10010|   d004|1996-11-24|2000-06-26|
+------+-------+----------+----------+
only showing top 10 rows



In [17]:
 # count distinct dept in dept_emp DF.

dept_emp.select(countDistinct("dept_no").alias("Distinct Department")).show()

+-------------------+
|Distinct Department|
+-------------------+
|                  9|
+-------------------+



**approx_count_distinct**
`approx_count_distinct`: It is used to count the approximate distinct count in the DataFrame. When the dataset is large it might consume more time to calculate distinct count, so we use `approx_count_distinct` function to count approximate distinct records. The additional parameter to this function is the maximum estimation error allowed during calculation.

In [18]:
# calculate approximate distinct count in employees DF

from pyspark.sql.functions import approx_count_distinct

employees.select(approx_count_distinct("emp_no")).show()

+-----------------------------+
|approx_count_distinct(emp_no)|
+-----------------------------+
|                       298930|
+-----------------------------+



In [19]:
# calculate approximate distinct count in employees DF
# with 0.1 maximum estimation erorr

from pyspark.sql.functions import approx_count_distinct

employees.select(approx_count_distinct("emp_no", 0.1)).show()

+-----------------------------+
|approx_count_distinct(emp_no)|
+-----------------------------+
|                       276091|
+-----------------------------+



**first**   
`first`: It is used to find the first records from a DataFrame. The value is selected based on rows in the DataFrame.

In [20]:

from pyspark.sql.functions import first

employees.select(first("emp_no"), first("hire_date")).show()

+--------------------+-----------------------+
|first(emp_no, false)|first(hire_date, false)|
+--------------------+-----------------------+
|               10001|             1986-06-26|
+--------------------+-----------------------+



**last**   
`last`: It is used to find the last records from a DataFrame. The value is selected based on rows in the DataFrame.

In [21]:

from pyspark.sql.functions import last

employees.select(last("emp_no"), last("hire_date")).show()

+-------------------+----------------------+
|last(emp_no, false)|last(hire_date, false)|
+-------------------+----------------------+
|             499999|            1997-11-30|
+-------------------+----------------------+



**min**   
`min`: It is used to find the minimum value from a DataFrame.

In [22]:
# find the minimum salary from salaries DF.

from pyspark.sql.functions import min

salaries.select(min("salary").alias("minimum_salary")).show()

+--------------+
|minimum_salary|
+--------------+
|         38623|
+--------------+



**max**   
`max`: It is used to find the maximum value from a DataFrame.

In [23]:
# find the maximun salary from salaries DF.

from pyspark.sql.functions import max

salaries.select(max("salary").alias("maximum_salary")).show()

+--------------+
|maximum_salary|
+--------------+
|        158220|
+--------------+



In [24]:
# find the minimum and maximum salary

salaries.select(min("salary").alias("min_salary"), max("salary").alias("max_salary")).show()

salaries.printSchema()

+----------+----------+
|min_salary|max_salary|
+----------+----------+
|     38623|    158220|
+----------+----------+

root
 |-- emp_no: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- from_date: date (nullable = true)
 |-- to_date: date (nullable = true)



In [25]:
# find the minimum and maximum value for emp_no and salary from salaries DF.
# Although there in no relation between two column i.e. emp_no and salary but to show different operation.

salaries.select(min("emp_no").alias("min_empNumber"),\
                max("emp_no").alias("max_empNumber"),\
                min("salary").alias("min_Salary"),\
                max("salary").alias("max_Salary")
        ).show()

+-------------+-------------+----------+----------+
|min_empNumber|max_empNumber|min_Salary|max_Salary|
+-------------+-------------+----------+----------+
|        10001|       499999|     38623|    158220|
+-------------+-------------+----------+----------+



**sum**   
`sum`: It is used to find the addition or summation of values in the column.

In [26]:
# find the sum of salary from salaries DF.

from pyspark.sql.functions import sum

salaries.select(sum("salary").alias("sum of salary ")).show()

+--------------+
|sum of salary |
+--------------+
|  181480757419|
+--------------+



**sumDistinct**   
`sumDistinct`: It is used to find the distinct addition or summation of values in the column.

In [27]:
# find the distinct sum of salary from salaries DF.

from pyspark.sql.functions import sumDistinct

salaries.select(sumDistinct("salary").alias("distinct sum of salary ")).show()

+-----------------------+
|distinct sum of salary |
+-----------------------+
|             7078688488|
+-----------------------+



In [28]:
# find the sum and distinct sum of salary from salaries DF.

salaries.select(sum("salary").alias("sum of salary "),\
                sumDistinct("salary").alias("distinct sum of salary ")
        ).show()

+--------------+-----------------------+
|sum of salary |distinct sum of salary |
+--------------+-----------------------+
|  181480757419|             7078688488|
+--------------+-----------------------+



**avg**   
`avg`: It is used to find the average values from the DataFrame. It is same as `mean()`. Averge can also be calcuated by `sum/count`.

In [29]:
# find the avg salary from salaries DF.

from pyspark.sql.functions import avg, mean

salaries.select(avg("salary").alias("Average Salary"),\
               mean("salary").alias("Mean Salary"),\
               sum("salary")/count("salary"))\
        .show()

+------------------+------------------+-----------------------------+
|    Average Salary|       Mean Salary|(sum(salary) / count(salary))|
+------------------+------------------+-----------------------------+
|63810.744836143705|63810.744836143705|           63810.744836143705|
+------------------+------------------+-----------------------------+



**Statistical Functions**

**corr**   
`corr` is used to measure the correlation between two columns. It measures the Pearson correlation coefficient ranged from -1 to +1.

In [30]:
# Find the correlation between salary and 10% increase of salary.

from pyspark.sql.functions import expr, corr

salaries.select(corr("salary", expr("(salary * 0.10) + salary"))).show()

+----------------------------------------+
|corr(salary, ((salary * 0.10) + salary))|
+----------------------------------------+
|                      1.0000000000000775|
+----------------------------------------+



**covar_pop**   
`covar_pop` is used to measure the population covariance.

In [31]:
# Find the population covariance between salary and 10% increase of salary.

from pyspark.sql.functions import covar_pop

salaries.select(covar_pop("salary", expr("(salary * 0.10) + salary"))).show()

+---------------------------------------------+
|covar_pop(salary, ((salary * 0.10) + salary))|
+---------------------------------------------+
|                          3.143505413914679E8|
+---------------------------------------------+



**covar_samp**   
`covar_samp` is used to measure the sample covariance.

In [32]:
# Find the sample covariance between salary and 10% increase of salary.

from pyspark.sql.functions import covar_samp

salaries.select(covar_samp("salary", expr("(salary * 0.10) + salary"))).show()

+----------------------------------------------+
|covar_samp(salary, ((salary * 0.10) + salary))|
+----------------------------------------------+
|                          3.1435065192081285E8|
+----------------------------------------------+



**variance**   
`variance` is used to find the sample variance. Variance is the average of squared difference from the mean. It is similar to `var_samp`.

In [33]:
# Find the variance of salary.

from pyspark.sql.functions import variance

salaries.select(variance("salary")).show()

+--------------------+
|    var_samp(salary)|
+--------------------+
|2.8577331992799246E8|
+--------------------+



**var_samp**   
`var_samp` is used to find the sample variance. Variance is the average of squared difference from the mean.

In [34]:
# Find the sample variance of salary.

from pyspark.sql.functions import var_samp

salaries.select(var_samp("salary")).show()

+--------------------+
|    var_samp(salary)|
+--------------------+
|2.8577331992799246E8|
+--------------------+



**var_pop**   
`var_pop` is used to find the population variance. 

In [35]:
# Find the population variance of salary.

from pyspark.sql.functions import var_pop

salaries.select(var_pop("salary")).show()

+-------------------+
|    var_pop(salary)|
+-------------------+
|2.857732194467698E8|
+-------------------+



**stddev**   
`stddev` is used to find the standard deviation.  It is similar to `stddev_samp`. Standard deviation is the square root of the variance.

In [36]:
# Find the standard deviation of salary.

from pyspark.sql.functions import stddev

salaries.select(stddev("salary")).show()

+-------------------+
|stddev_samp(salary)|
+-------------------+
| 16904.831259968036|
+-------------------+



**stddev_samp**   
`stddev_samp` is used to find the sample standard deviation.

In [37]:
# Find the sample standard deviation of salary.

from pyspark.sql.functions import stddev_samp

salaries.select(stddev_samp("salary")).show()

+-------------------+
|stddev_samp(salary)|
+-------------------+
| 16904.831259968036|
+-------------------+



**stddev_pop**   
`stddev_pop` is used to find the population standard deviation.

In [38]:
# Find the population standard deviation of salary.

from pyspark.sql.functions import stddev_pop

salaries.select(stddev_pop("salary")).show()

+------------------+
|stddev_pop(salary)|
+------------------+
|16904.828288000142|
+------------------+



**skewness**   
`skewness` describes the assymmetry in a random variable's probability distribution. It measures the asymmetry of values of data around the mean.

In [39]:
# Find the skewness of salary.

from pyspark.sql.functions import skewness

salaries.select(skewness("salary")).show()

+------------------+
|  skewness(salary)|
+------------------+
|0.7779492524394536|
+------------------+



**kurtosis**   
`kurtosis` describes the realtive peakedness or flatness of a distribution compared with the normal distribution. It measure the tail of data.

In [40]:
# Find the kurtosis of salary.

from pyspark.sql.functions import kurtosis

salaries.select(kurtosis("salary")).show()

+------------------+
|  kurtosis(salary)|
+------------------+
|0.2919969859460094|
+------------------+



#### 4.3.2 GroupBy

Grouping is used to aggregate data based on particular groups in the dataset. It is mostly used for categorical data compared to continuous data. The data are first grouped on particular column(s) and aggregation is applied. The step for group aggregation is shown below:   
1. groupBy("columns") : It returns RelationalGroupedDataset
2. aggregate function : It returns DataFrame

Example 1: Count total employees for each gender group.      
`employees.groupBy("gender").count().show()`   
Example 2: Count total salary for each group/category.     
`salaries.groupBy("salary").count().show()`   
Example 3: Find total employee working in each department.      
`dept_emp.groupBy("dept_no").count().show()`   

Figure 4.3.2 illustrates the aggregate with grouping for example 3.

![Figure: 4.3.2 Aggregation with grouping](grouping_aggregation.png)

In [41]:
dept_emp.printSchema()

root
 |-- emp_no: integer (nullable = true)
 |-- dept_no: string (nullable = true)
 |-- from_date: date (nullable = true)
 |-- to_date: date (nullable = true)



In [42]:
# groupBy returning GroupedData

dept_emp.groupBy("dept_no")

In [43]:
# aggregate function returning DataFrame after applying on GroupedData

dept_emp.groupBy("dept_no").count()

DataFrame[dept_no: string, count: bigint]

In [44]:
# Find total employee working in each department.

dept_emp.groupBy("dept_no").count().show()

+-------+-----+
|dept_no|count|
+-------+-----+
|   d005|85707|
|   d009|23580|
|   d003|17786|
|   d001|20211|
|   d007|52245|
|   d004|73485|
|   d002|17346|
|   d006|20117|
|   d008|21126|
+-------+-----+



In [45]:
# Count total employees for each gender group.     

employees.groupBy("gender").count().show() 

+------+------+
|gender| count|
+------+------+
|     F|120051|
|     M|179973|
+------+------+



In [46]:
# Find total employee working on each department starting on same year.

from pyspark.sql.functions import year
dept_emp.groupBy("dept_no", year("from_date")).count().show(30)

+-------+---------------+-----+
|dept_no|year(from_date)|count|
+-------+---------------+-----+
|   d005|           1997| 5768|
|   d006|           2002|  205|
|   d002|           1992| 1119|
|   d006|           2001|  335|
|   d008|           1993| 1344|
|   d005|           1999| 5740|
|   d007|           2000|  492|
|   d007|           1991| 3342|
|   d006|           1988| 1218|
|   d002|           1998| 1140|
|   d006|           1998| 1533|
|   d003|           2001|   82|
|   d004|           1989| 4556|
|   d008|           1996| 1449|
|   d007|           1985| 3090|
|   d004|           1993| 4714|
|   d003|           1995| 1205|
|   d007|           1998| 3477|
|   d003|           1987| 1178|
|   d003|           1986| 1117|
|   d002|           1995| 1163|
|   d003|           1996| 1231|
|   d004|           1998| 5102|
|   d006|           1986| 1109|
|   d001|           1995| 1238|
|   d001|           1997| 1394|
|   d003|           1988| 1147|
|   d001|           1998| 1480|
|   d008

**Using Expressions in Grouping**   

Instead of passing aggregate function into `select` statement as expression, we can use `agg` method to accomplish same task. It allows to pass aggregate function as expression using `expr` as well as aliasing the column name. The code looks more clear and easier to undertand by using `agg` method.

In [47]:
# Count total employees for each gender group.     

employees.groupBy("gender")\
         .agg(count("gender").alias("Total Count"))\
         .show() 

+------+-----------+
|gender|Total Count|
+------+-----------+
|     F|     120051|
|     M|     179973|
+------+-----------+



In [48]:
# Find total employee working on each department starting on same year.

from pyspark.sql.functions import year

dept_emp.groupBy("dept_no", year("from_date"))\
        .agg(count("dept_no").alias("Total Count"),
        expr("count(dept_no)").alias("Total Count"))\
        .show()

+-------+---------------+-----------+-----------+
|dept_no|year(from_date)|Total Count|Total Count|
+-------+---------------+-----------+-----------+
|   d005|           1997|       5768|       5768|
|   d006|           2002|        205|        205|
|   d002|           1992|       1119|       1119|
|   d006|           2001|        335|        335|
|   d008|           1993|       1344|       1344|
|   d005|           1999|       5740|       5740|
|   d007|           2000|        492|        492|
|   d007|           1991|       3342|       3342|
|   d006|           1988|       1218|       1218|
|   d002|           1998|       1140|       1140|
|   d006|           1998|       1533|       1533|
|   d003|           2001|         82|         82|
|   d004|           1989|       4556|       4556|
|   d008|           1996|       1449|       1449|
|   d007|           1985|       3090|       3090|
|   d004|           1993|       4714|       4714|
|   d003|           1995|       1205|       1205|


#### 4.3.3 Window

Window function operate on a set of rows and return a single value for each row. Window defines the set of rows for which the function operates. This function calculates a return values for all the input records based on certain group of rows known as `frame`.   
When we use `groupby` method, all the row will be categoried into only one group. Using window functions, we can categorize row into one or more frames. Spark has three types of window function:   
* ranking functions: rank, denseRank, percentRank, ntile, rowNumber
* analytics functions: cumeDist, firstValue, lastValue, lag, lead
* aggregate functions: avg, count, collect_list

[More about Spark Window Functions](https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html)

Window function uses:   
* `partitionBy`: used to subdivide the window into partitions. Partition in window function means split or specifying data on certain group. So don't be confused with the partition describe on prior chapter.   
* `orderBy`: used to order the partitioned data.

For example: Find the average salary for each year.

In [49]:
# Create salDF by adding hired_year column from salaries DF.

salDF = salaries.withColumn("hired_year", year("from_date"))

In [50]:
salDF.printSchema()
salDF.show(10)

root
 |-- emp_no: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- from_date: date (nullable = true)
 |-- to_date: date (nullable = true)
 |-- hired_year: integer (nullable = true)

+------+------+----------+----------+----------+
|emp_no|salary| from_date|   to_date|hired_year|
+------+------+----------+----------+----------+
| 10001| 60117|1986-06-26|1987-06-26|      1986|
| 10001| 62102|1987-06-26|1988-06-25|      1987|
| 10001| 66074|1988-06-25|1989-06-25|      1988|
| 10001| 66596|1989-06-25|1990-06-25|      1989|
| 10001| 66961|1990-06-25|1991-06-25|      1990|
| 10001| 71046|1991-06-25|1992-06-24|      1991|
| 10001| 74333|1992-06-24|1993-06-24|      1992|
| 10001| 75286|1993-06-24|1994-06-24|      1993|
| 10001| 75994|1994-06-24|1995-06-24|      1994|
| 10001| 76884|1995-06-24|1996-06-23|      1995|
+------+------+----------+----------+----------+
only showing top 10 rows



In [51]:
# Find the average salary for each year.
from pyspark.sql.functions import avg

salDF.groupBy("hired_year").agg(avg("salary")).show()

+----------+------------------+
|hired_year|       avg(salary)|
+----------+------------------+
|      1990|  57839.4601216819|
|      1988| 55862.44652517686|
|      1997| 64565.42639478537|
|      1994|61727.758915310624|
|      1991| 58803.86967667994|
|      1989|56840.672790937606|
|      1996|  63618.9425859676|
|      1998| 65540.26833919891|
|      1985|53182.358005794566|
|      1987| 54959.62837743732|
|      1995| 62681.04298318265|
|      2001|  70694.9158819634|
|      1992|59758.741593412415|
|      2000| 68556.27813593448|
|      1986|54084.778591564136|
|      1999| 66525.36188720747|
|      2002| 72683.93965798624|
|      1993| 60753.65652228216|
+----------+------------------+



In [52]:
# Find the average salary for each year.

from pyspark.sql.window import Window

# create partition by year using Window Function
salWindow = Window.partitionBy("hired_year")
# apply aggregate function over partitioned data
avgSal = avg(col("salary")).over(salWindow)
# select and display DataFrame
salDF.select("*",avgSal.alias("Average Salary")).show(20)

+------+------+----------+----------+----------+----------------+
|emp_no|salary| from_date|   to_date|hired_year|  Average Salary|
+------+------+----------+----------+----------+----------------+
| 10001| 66961|1990-06-25|1991-06-25|      1990|57839.4601216819|
| 10004| 48271|1990-11-30|1991-11-30|      1990|57839.4601216819|
| 10005| 82621|1990-09-12|1991-09-12|      1990|57839.4601216819|
| 10006| 40000|1990-08-05|1991-08-05|      1990|57839.4601216819|
| 10007| 60740|1990-02-10|1991-02-10|      1990|57839.4601216819|
| 10009| 70889|1990-02-17|1991-02-17|      1990|57839.4601216819|
| 10011| 42365|1990-01-22|1991-01-22|      1990|57839.4601216819|
| 10013| 46305|1990-10-19|1991-10-19|      1990|57839.4601216819|
| 10018| 61648|1990-04-02|1991-04-02|      1990|57839.4601216819|
| 10021| 59700|1990-02-09|1991-02-09|      1990|57839.4601216819|
| 10025| 50120|1990-08-16|1991-08-16|      1990|57839.4601216819|
| 10032| 48426|1990-06-20|1991-06-20|      1990|57839.4601216819|
| 10033| 5

In [53]:
# Find the average salary for each year.

from pyspark.sql.window import Window

# create partition by year using Window Function
salWindow = Window.partitionBy("hired_year")

salDF.withColumn("Average Salary",avg(col("salary")).over(salWindow)).show(20)

+------+------+----------+----------+----------+----------------+
|emp_no|salary| from_date|   to_date|hired_year|  Average Salary|
+------+------+----------+----------+----------+----------------+
| 10001| 66961|1990-06-25|1991-06-25|      1990|57839.4601216819|
| 10004| 48271|1990-11-30|1991-11-30|      1990|57839.4601216819|
| 10005| 82621|1990-09-12|1991-09-12|      1990|57839.4601216819|
| 10006| 40000|1990-08-05|1991-08-05|      1990|57839.4601216819|
| 10007| 60740|1990-02-10|1991-02-10|      1990|57839.4601216819|
| 10009| 70889|1990-02-17|1991-02-17|      1990|57839.4601216819|
| 10011| 42365|1990-01-22|1991-01-22|      1990|57839.4601216819|
| 10013| 46305|1990-10-19|1991-10-19|      1990|57839.4601216819|
| 10018| 61648|1990-04-02|1991-04-02|      1990|57839.4601216819|
| 10021| 59700|1990-02-09|1991-02-09|      1990|57839.4601216819|
| 10025| 50120|1990-08-16|1991-08-16|      1990|57839.4601216819|
| 10032| 48426|1990-06-20|1991-06-20|      1990|57839.4601216819|
| 10033| 5

In [54]:
# Find the average, total, minimium, and maximum salary for each year.

from pyspark.sql.window import Window
from pyspark.sql.functions import col

# create partition by year using Window Function
salWindow = Window.partitionBy("hired_year")

salDF.withColumn("Average Salary", avg(col("salary")).over(salWindow))\
     .withColumn("Total Salary", sum(col("salary")).over(salWindow))\
     .withColumn("Minimun Salary", min(col("salary")).over(salWindow))\
     .withColumn("Maximum Salary", max(col("salary")).over(salWindow)).show()

+------+------+----------+----------+----------+----------------+------------+--------------+--------------+
|emp_no|salary| from_date|   to_date|hired_year|  Average Salary|Total Salary|Minimun Salary|Maximum Salary|
+------+------+----------+----------+----------+----------------+------------+--------------+--------------+
| 10001| 66961|1990-06-25|1991-06-25|      1990|57839.4601216819|  6626146391|         38851|        134572|
| 10004| 48271|1990-11-30|1991-11-30|      1990|57839.4601216819|  6626146391|         38851|        134572|
| 10005| 82621|1990-09-12|1991-09-12|      1990|57839.4601216819|  6626146391|         38851|        134572|
| 10006| 40000|1990-08-05|1991-08-05|      1990|57839.4601216819|  6626146391|         38851|        134572|
| 10007| 60740|1990-02-10|1991-02-10|      1990|57839.4601216819|  6626146391|         38851|        134572|
| 10009| 70889|1990-02-17|1991-02-17|      1990|57839.4601216819|  6626146391|         38851|        134572|
| 10011| 42365|1990

In [55]:
salDF.withColumn("Average Salary",avg(col("salary")).over(salWindow))\
     .withColumn("Total Salary",sum(col("salary")).over(salWindow))\
     .withColumn("Minimun Salary",min(col("salary")).over(salWindow))\
     .withColumn("Maximum Salary",max(col("salary")).over(salWindow)).where(col("hired_year") == 1991).show()

+------+------+----------+----------+----------+-----------------+------------+--------------+--------------+
|emp_no|salary| from_date|   to_date|hired_year|   Average Salary|Total Salary|Minimun Salary|Maximum Salary|
+------+------+----------+----------+----------+-----------------+------------+--------------+--------------+
| 10001| 71046|1991-06-25|1992-06-24|      1991|58803.86967667994|  7798804412|         39115|        134624|
| 10004| 50594|1991-11-30|1992-11-29|      1991|58803.86967667994|  7798804412|         39115|        134624|
| 10005| 83735|1991-09-12|1992-09-11|      1991|58803.86967667994|  7798804412|         39115|        134624|
| 10006| 42085|1991-08-05|1992-08-04|      1991|58803.86967667994|  7798804412|         39115|        134624|
| 10007| 62745|1991-02-10|1992-02-10|      1991|58803.86967667994|  7798804412|         39115|        134624|
| 10009| 71434|1991-02-17|1992-02-17|      1991|58803.86967667994|  7798804412|         39115|        134624|
| 10011| 4

#### 4.3.4 Grouping Set   
`Grouping Set` is used for complex aggregations. For example, if we want to aggregate data based on (salary, year), (salary), and (year) in a single statement then `grouping set` is used. It is only availabe in Spark SQL. To apply in DataFrame API, we need to use `rollup` and `cube`.   

For example: Find the total project based on set of (team, language), (team) and (language). 

SELECT team, language, sum(project_id) FROM projects GROUP BY GROUPING SETS ((team, language), (team), (language));

| team | language | sum(project) |   
| ------- | -------- | ------------ |
| A  | Python | 15 |
| B  | Java | 2 |   
| B  | Scala | 1 |   
| C  | Python | 5 |   
| C  | Scala | 9 |   
| A  | Scala | 3 |   
| B  | Python | 10 |     
| C  || 14 |   
| A  || 18 |   
| B  || 13 |      
|| Java  | 2 |   
|| Python | 30 |   
|| Scala | 13 |    

#### 4.3.4.1 Rollup    
`rollup` is used to perform multi-dimensional aggregation over combination of groups of specified columns. It returns RelationalGroupedDataset where aggregate function can be applied. For example, if we want to perform rollup on *(col1, col2, col3)*, it will compute subtotal for the combinations *(col1, col2, col3), (col2, col3) and (col1)*. Basically, rollup is the extension of `groupBy`.

For example: Find the total salary for all hire_date and dept_name. `rollup` performs aggregation over group of  hired_date and dept_name.

In [61]:
highest_salary_employee.printSchema()

root
 |-- id: long (nullable = true)
 |-- emp_no: long (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- salary: long (nullable = true)
 |-- from_date_x: date (nullable = true)
 |-- to_date_x: date (nullable = true)
 |-- dept_no: string (nullable = true)
 |-- from_date_y: date (nullable = true)
 |-- to_date_y: date (nullable = true)
 |-- dept_name: string (nullable = true)



In [62]:
# Find the total salary for all hire_date and dept_name.

highest_salary_employee.rollup("hire_date", "dept_name").agg(sum("salary"))\
                       .selectExpr("hire_date", "dept_name", "`sum(salary)` as total_sum")\
                       .orderBy("hire_date").show(30)           

+----------+----------------+---------+
| hire_date|       dept_name|total_sum|
+----------+----------------+---------+
|      null|            null|  2205189|
|1985-09-17|            null|    97338|
|1985-09-17|           Sales|    97338|
|1985-10-14|            null|   192644|
|1985-10-14|     Development|    96322|
|1985-10-14|        Research|    96322|
|1986-02-11|           Sales|   105565|
|1986-02-11|            null|   105565|
|1986-02-26|            null|   103672|
|1986-02-26|     Development|   103672|
|1986-03-01|            null|   104747|
|1986-03-01|           Sales|   104747|
|1986-09-08|           Sales|   102651|
|1986-09-08|            null|   102651|
|1986-11-05|           Sales|    97077|
|1986-11-05|            null|    97077|
|1986-11-16|            null|   100182|
|1986-11-16|     Development|   100182|
|1988-09-02|Customer Service|    98003|
|1988-09-02|            null|   196006|
|1988-09-02|           Sales|    98003|
|1988-10-18|           Sales|    98538|


#### 4.3.4.2 Cube    
`cube` is used to perform multi-dimensional aggregation across every permutation of specified columns. It returns RelationalGroupedDataset where aggregate function can be applied. `cube` perform more than `rollup`. It perform `rollup` with aggregation over all the missing combination of specified columns. The `nulls` values in all specified column gives the grand total across those columns. Basically, cube is an extension of `rollup`.
For example, if we want to perform cube on *(col1, col2, col3)*, it will compute subtotal for the combinations *(col1, col2), (col1, col3), (col1), (col2, col3), (col2) and (col3)*.


For example: Find the total salary across all hire_date and dept_name. `cube` performs aggregation for both hired_date and dept_name.

In [63]:
# Find the total salary across all hire_date and dept_name.

highest_salary_employee.cube("hire_date", "dept_name").agg(sum("salary"))\
                       .selectExpr("hire_date", "dept_name", "`sum(salary)` as total_sum")\
                       .orderBy("hire_date").show(30)  

+----------+----------------+---------+
| hire_date|       dept_name|total_sum|
+----------+----------------+---------+
|      null| Human Resources|    96471|
|      null|         Finance|   302829|
|      null|        Research|    96322|
|      null|      Production|    96646|
|      null|       Marketing|    99651|
|      null|           Sales|  1115091|
|      null|     Development|   300176|
|      null|            null|  2205189|
|      null|Customer Service|    98003|
|1985-09-17|           Sales|    97338|
|1985-09-17|            null|    97338|
|1985-10-14|        Research|    96322|
|1985-10-14|     Development|    96322|
|1985-10-14|            null|   192644|
|1986-02-11|            null|   105565|
|1986-02-11|           Sales|   105565|
|1986-02-26|     Development|   103672|
|1986-02-26|            null|   103672|
|1986-03-01|            null|   104747|
|1986-03-01|           Sales|   104747|
|1986-09-08|           Sales|   102651|
|1986-09-08|            null|   102651|


**grouping_id**    
`grouping_id` is used to specify level of aggregation while performing `cubes` and `rollups`. It helps to filter the aggregation by specifying the id.

The grouping id generated for the code snippet below is shown in table.

| Grouping ID | Description |
| ----------- | ----------- |
| 0 | Total sum of *hired_date* and *dept_name* | 
| 1 | Total sum for *hired_date* |
| 2 | Total sum per *dept_name* |
| 3 | Total sum regardless of *hire_date* and *dept_name* |

In [64]:
from pyspark.sql.functions import grouping_id

highest_salary_employee.cube("hire_date", "dept_name").agg(grouping_id(), sum("salary"))\
                       .orderBy("hire_date").show(50)  

+----------+----------------+-------------+-----------+
| hire_date|       dept_name|grouping_id()|sum(salary)|
+----------+----------------+-------------+-----------+
|      null|     Development|            2|     300176|
|      null|            null|            3|    2205189|
|      null| Human Resources|            2|      96471|
|      null|           Sales|            2|    1115091|
|      null|       Marketing|            2|      99651|
|      null|         Finance|            2|     302829|
|      null|        Research|            2|      96322|
|      null|      Production|            2|      96646|
|      null|Customer Service|            2|      98003|
|1985-09-17|           Sales|            0|      97338|
|1985-09-17|            null|            1|      97338|
|1985-10-14|     Development|            0|      96322|
|1985-10-14|            null|            1|     192644|
|1985-10-14|        Research|            0|      96322|
|1986-02-11|           Sales|            0|     

#### 4.4 Pivot       
`pivot` is used to convert row into a column.

In [65]:
titles.printSchema()
titles.show(10)
titles.groupby("from_date").pivot("title").sum().show(20)

root
 |-- emp_no: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- from_date: date (nullable = true)
 |-- to_date: date (nullable = true)

+------+---------------+----------+----------+
|emp_no|          title| from_date|   to_date|
+------+---------------+----------+----------+
| 10001|Senior Engineer|1986-06-26|9999-01-01|
| 10002|          Staff|1996-08-03|9999-01-01|
| 10003|Senior Engineer|1995-12-03|9999-01-01|
| 10004|       Engineer|1986-12-01|1995-12-01|
| 10004|Senior Engineer|1995-12-01|9999-01-01|
| 10005|   Senior Staff|1996-09-12|9999-01-01|
| 10005|          Staff|1989-09-12|1996-09-12|
| 10006|Senior Engineer|1990-08-05|9999-01-01|
| 10007|   Senior Staff|1996-02-11|9999-01-01|
| 10007|          Staff|1989-02-10|1996-02-11|
+------+---------------+----------+----------+
only showing top 10 rows

+----------+------------------+--------+-------+---------------+------------+-------+----------------+
| from_date|Assistant Engineer|Engineer|Manager|Senior 

#### 4.5 User Defined Aggregate Functions    
User-Defined Aggregate Function (UDAF) is a custom built aggregate function. This function calculates over group of records compared to single records in UDF. Spark uses `AggregationBuffer` to store intermediate resultset during aggregate function.   

To creating UDAF:   
* Inherit `UserDefinedAggregateFunction` class 
* Implement following methods:    
    * `inputSchema`
    * `bufferSchema`
    * `dataType`
    * `deterministic`
    * `initialize`
    * `update`
    * `merge`
    * `evaluate`